In [ ]:
import luigi
import pandas as pd
import os
import logging



Création du dossier pour stocker les données nétoyer

In [ ]:
# Vérifiez si le dossier n'existe pas déjà

import shutil


dossier = 'CleanData'

if not os.path.exists(dossier):
    # Créez le dossier
    os.makedirs(dossier, mode=0o777)
    print("Dossier créé avec succès.")


In [ ]:
import sys

class ReadDataCanton(luigi.Task):
    file_path = luigi.Parameter()


    def output(self):
        return luigi.LocalTarget("./CleanData/canton.csv")

    def run(self):
        
        df = pd.read_csv(self.file_path, sep=";")
        cantons = pd.DataFrame(columns=df.columns[0:25])
        df_partis = pd.read_csv('./RawData/Partis.csv', sep=",")

        for index, row in df.iterrows():
            #print(f"index : {index} \n row : {row[0:18]} \n\n")
            row1 = pd.Series(row[0:18], index=df.columns[0:18])

            for i in range(0, 10):
                # Ligne 2
                row2 = pd.Series(row.iloc[18 + i * 7:18 + ((i + 1) * 7)])

                df_row2 = pd.DataFrame([row2])
                
                df_row2_renamed = pd.DataFrame(df_row2.values, columns=['N°Panneau','Sexe','Nom','Prenom','Voix','% Voix/Ins','% Voix/Exp'])

                # Fusionner les deux lignes
                merged_row = pd.concat([row1, df_row2_renamed.squeeze()], axis=0)

                # Convertir la ligne fusionnée en DataFrame avec une seule ligne
                merged_row_df = pd.DataFrame([merged_row])

                # Ajouter la ligne fusionnée au DataFrame
                cantons = pd.concat([cantons, merged_row_df], ignore_index=True)

                #print(len(cantons))

        cantons = pd.merge(cantons, df_partis, on=['Nom', 'Prenom'], how='left')

        print(cantons.duplicated(['Nom', 'Prenom']).sum())
        print(df_partis.duplicated(['Nom', 'Prenom']).sum())

        #display(cantons)
        cantons.to_csv(self.output().path, index=False)




In [ ]:
class ExtractCandidats(luigi.Task):
    file_path = luigi.Parameter()

    def requires(self):
        return ReadDataCanton(file_path="./RawData/canton_t1_2017.csv")
    
    def output(self):
        return luigi.LocalTarget("./CleanData/candidats.csv")

    def run(self):
                
        df = pd.read_csv(self.file_path, sep=",")
       
        display(df);

        df = df.loc[:, ["Prenom", "Nom"]]
        df = df.drop_duplicates()
        
        df.to_csv(self.output().path, index=False)

In [ ]:
class ExtractDelinquance(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/delinquance.csv")

    def run(self):
                        
        df = pd.read_csv(self.file_path, sep=";")
        # Faites quelque chose avec le dataframe, par exemple, l'afficher

        df.to_csv(self.output().path, index=False)

In [ ]:
class ReadAllData(luigi.Task):
    def requires(self):
        return [ExtractCandidats(file_path="./CleanData/canton.csv"), 
                ExtractDelinquance(file_path="./RawData/Donnees_delinquance.csv")]
    
    def run(self):
        print("lancement")

    def output(self):
        return luigi.LocalTarget('result.txt')

In [ ]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')

dossier = "./CleanData/"
Restart = True

if Restart is True:
    # Parcourir tous les fichiers du dossier
    for fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, fichier)
        # Supprimer le fichier
        os.remove(chemin_fichier)

luigi.build([ReadAllData()], local_scheduler=False, no_lock=True)